In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICE'] = ''
import random

%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
from omegaconf import OmegaConf

%load_ext autoreload
%autoreload 2

from importlib.util import find_spec
if find_spec("text_recognizer") is None:
    import sys
    sys.path.append('..')

In [ ]:
from hydra import compose, initialize
from omegaconf import OmegaConf
from hydra.utils import instantiate

In [ ]:
path = "../training/conf/network/efficientnet.yaml"

In [ ]:
with open(path, "rb") as f:
    cfg = OmegaConf.load(f)

In [ ]:
cfg

In [ ]:
cfg.depth = 4

In [ ]:
net = instantiate(cfg)

In [ ]:
from torchinfo import summary

In [ ]:
summary(net, (2, 1, 56, 1024), device="cpu", depth=4)